In [1]:
import sys
sys.path.insert(1, '/home/taylor/Classes/cs230/achtung')
import achtung;

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
import gym
import numpy as np
import pickle

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import VecTransposeImage
# from stable_baselines3.common.atari_wrappers import AtariWrapper

In [5]:
env = achtung.Achtung()
env.speed = 0 # set to zero for training (i.e., no frame delay)
env.render_game = False

Achtung Die Kurve!


In [6]:
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 3)
env = VecTransposeImage(env)

env.observation_space

Box(0, 255, (3, 80, 80), uint8)

In [7]:
model = DQN("CnnPolicy", 
            env, 
            buffer_size=10000,
            learning_rate=1e-4,
            batch_size=32,
            learning_starts=100000,
            target_update_interval=1000,
            train_freq=4,
            gradient_steps=1,
            exploration_fraction=0.1,
            exploration_final_eps=0.01)

In [8]:
# Evaluate the initial random policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=25)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:30.40 +/- 14.02


In [ ]:
# Train
rewards = []
stds = []
for i in range(1):
    print("iteration: ", i+1)
    model.learn(total_timesteps=10000)
    print("   saving...")
    model.save("dqn_achtung")
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=25, deterministic=True)
    print(f"   mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    rewards.append(mean_reward)
    stds.append(std_reward)
    
     
    with open("baselines_training/dqn_reward.txt", "wb") as f:   
        pickle.dump(rewards, f)
    with open("baselines_training/dqn_std.txt", "wb") as f:   
        pickle.dump(stds, f)

iteration:  1


In [ ]:
# Evaluate the trained policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")